# Case Study

> 3.1.5 장에 해당하는 코드

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
# 2.2.8 장에서 정의한 훈련 및 테스트 함수를 불러온다.
from train_utils import train, test, main

# 데이터 셋 불러오기
train_dataset = datasets.CIFAR10('./data', train=True, download=True, 
                                 transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10('./data', train=False, 
                                transform=transforms.ToTensor())

# 환경 변수 설정
BATCH = 128  # 미니배치크기
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # 디바이스
STEP = 30  # 총 반복스텝
PRINT_STEP = 180  # 경과 print 기간

# 데이터로더 선언
train_loader = DataLoader(train_dataset, batch_size=BATCH, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH, shuffle=True)

Files already downloaded and verified


## AlexNet

In [2]:
# 코드 3-13

class AlexNet(nn.Module):
    def __init__(self, n_in=32, output_size=10):
        super(AlexNet, self).__init__()
        """
        config:
        - kernel: c_out, k, s, p
        - pool: maxpool, (kernel_size=3, stride=2, padding=0)
        """
        self.n_in = n_in
        config = [(96, 11, 4, 0), "M", (256, 5, 1, 2), "M", 
                  (384, 3, 1, 1), (384, 3, 1, 1), (256, 3, 1, 1), "M"]
        
        self.convs = self._make_layers(config)
        self.fc = nn.Sequential(
                    nn.Linear(self._get_fc_input(config), 4096),
                    nn.Linear(4096, 4096),
                    nn.Linear(4096, output_size),
                    )

    def forward(self, x):
        out = self.convs(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

    def _make_layers(self, config):
        layers = []
        ch_in = 3
        for x in config:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=3, stride=2)]
            else:
                ch_out, k, s, p = x
                layers += [nn.Conv2d(ch_in, ch_out, 
                                     kernel_size=k, stride=s, padding=p),
                           nn.ReLU(inplace=True)]
                ch_in = ch_out
        return nn.Sequential(*layers)
    
    def _get_fc_input(self, config):
        n_in = self.n_in
        for x in config:
            if x == "M":
                pool_k, pool_s = 3, 2
                pool_n_out = (n_in - pool_k)/pool_s + 1
                n_in = pool_n_out
            else:
                ch_out, k, s, p = x
            
                conv_n_out = (n_in + 2*p - k)/s + 1
                n_in = conv_n_out
            
        fc_input = int(ch_out*n_in*n_in)
        return fc_input

## VGG Net

In [3]:
# 코드 3-14

class VGG(nn.Module):
    def __init__(self, vgg_name, n_in=32, output_size=10):
        super(VGG, self).__init__()
        """reference: https://github.com/kuangliu/pytorch-cifar"""
        self.n_in = n_in
        config = {
            'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
            'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
            'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
            'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
        }
        
        self.convs = self._make_layers(config[vgg_name])
        self.fc = nn.Linear(self._get_fc_input(config[vgg_name]), output_size)

    def forward(self, x):
        out = self.convs(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

    def _make_layers(self, config):
        layers = []
        in_channels = 3
        for x in config:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        return nn.Sequential(*layers)
    
    def _get_fc_input(self, config):
        n_in = self.n_in
        for x in config:
            if x == "M":
                pool_k, pool_s = 2, 2
                pool_n_out = (n_in - pool_k)/pool_s + 1
                n_in = pool_n_out
            else:
                ch_out = x
                k, s, p = 3, 1, 1
                conv_n_out = (n_in + 2*p - k)/s + 1
                n_in = conv_n_out
            
        fc_input = int(ch_out*n_in*n_in)
        return fc_input

In [4]:
# 모델 선언
torch.manual_seed(70)
model = VGG(vgg_name="VGG16", n_in=32, output_size=10).to(DEVICE)
# 손실함수 선언
loss_function = nn.CrossEntropyLoss()
# 옵티마이저 선언
optimizer = optim.Adam(model.parameters()) 

main(model=model, 
     train_loader=train_loader, 
     test_loader=test_loader, 
     loss_func=loss_function, 
     optimizer=optimizer, 
     n_step=STEP,
     device=DEVICE,
     save_path="cifar10_vgg16.pt", 
     print_step=PRINT_STEP)

Train Step: 1 (00.00%)  	Loss: 2.5153
Train Step: 1 (46.08%)  	Loss: 1.3035
Train Step: 1 (92.16%)  	Loss: 1.0016
Test set: Average loss: 1.4983, Accuracy: 5158/10000 (51.58%)

Train Step: 2 (00.00%)  	Loss: 0.9769
Train Step: 2 (46.08%)  	Loss: 1.0267
Train Step: 2 (92.16%)  	Loss: 0.9284
Test set: Average loss: 0.9764, Accuracy: 6552/10000 (65.52%)
discard previous state, best model state saved!

Train Step: 3 (00.00%)  	Loss: 0.7892
Train Step: 3 (46.08%)  	Loss: 0.7350
Train Step: 3 (92.16%)  	Loss: 0.6416
Test set: Average loss: 0.6630, Accuracy: 7779/10000 (77.79%)
discard previous state, best model state saved!

Train Step: 4 (00.00%)  	Loss: 0.5832
Train Step: 4 (46.08%)  	Loss: 0.4396
Train Step: 4 (92.16%)  	Loss: 0.5759
Test set: Average loss: 0.7363, Accuracy: 7535/10000 (75.35%)

Train Step: 5 (00.00%)  	Loss: 0.3670
Train Step: 5 (46.08%)  	Loss: 0.4801
Train Step: 5 (92.16%)  	Loss: 0.3762
Test set: Average loss: 0.5723, Accuracy: 8091/10000 (80.91%)
discard previous sta